# Visualizing bioinformatics data with plot.ly

## Introduction
This notebook is based on [An Introduction to Bioinformatics](http://readiab.org/). This notebook will follow the progression of the textbook but the textbook should be consulted for more detailed explanations of the analyses.  

This note book will cover the fundamentals of bioinformatics - how to interact with with raw sequence data, sequence alignment, and tree building. These are the most basic steps in bioinformatics and most analyses are based on these, or similar processes.

This notebook will cover:
1. Sequence Quality
2. Sequence Alignment
3. Tree building



### The dataset
The dataset we will be using is from the paper [Geography and Location Are the Primary Drivers of Office Microbiome Composition](http://msystems.asm.org/content/1/2/e00022-16). The data represented is raw sequence data produced by an illumina MiSeq of the bacterial 16S rRNA gene. The sequences present are from a single sample that was taken from the floor of an office in Flagstaff Arizona.

### Obtain the files

In [1]:
!wget https://github.com/johnchase/plotly-notebook/raw/master/raw_data.tar.gz
!tar -xvzf raw_data.tar.gz

--2016-11-27 18:06:03--  https://github.com/johnchase/plotly-notebook/raw/master/raw_data.tar.gz
Resolving github.com... 192.30.253.113, 192.30.253.112
Connecting to github.com|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/johnchase/plotly-notebook/master/raw_data.tar.gz [following]
--2016-11-27 18:06:03--  https://raw.githubusercontent.com/johnchase/plotly-notebook/master/raw_data.tar.gz
Resolving raw.githubusercontent.com... 151.101.40.133
Connecting to raw.githubusercontent.com|151.101.40.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2335320 (2.2M) [application/octet-stream]
Saving to: ‘raw_data.tar.gz.6’

raw_data.tar.gz.6   100%[===================>]   2.23M  2.41MB/s    in 0.9s    

2016-11-27 18:06:04 (2.41 MB/s) - ‘raw_data.tar.gz.6’ saved [2335320/2335320]

x ./._msa10.fna
x msa10.fna
x ./._seqs_quals.fastq
x seqs_quals.fastq


### Load the necessary libraries

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import colorlover as cl

import skbio
from skbio.alignment import global_pairwise_align_nucleotide
from skbio.sequence import DNA
import pandas as pd
import itertools
import numpy as np

In [20]:
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [3]:
py.sign_in('johnchase', '')


## 1. Sequence Quality
Because the quality of sequence data produced by high throughput sequencing varies between sequencing runs and samples it is important to look at the sequence quality and possibly filter or trim sequences, or remove samples where the quality is low. A more detailed description of the fastq format and quality scores can be found [here](http://scikit-bio.org/docs/latest/generated/skbio.io.format.fastq.html?highlight=fastq#module-skbio.io.format.fastq). Quality scores themselves are difficult to interpret, so we will use scikit-bio to decode the scores for us. Here we load the sequence data into a generator with scikit-bio.

In [5]:
f = '/home/paperspace/data/biology/bioinfo/seqs_quals.fastq'
seqs = skbio.io.read(f, format='fastq', verify=False, variant='illumina1.8')

We can view one of the `skbio.sequence._sequence.Sequence` entries in the generator object. This will display the sequence data and associated metadata. 

In [7]:
seq1 = seqs.__next__()
seq1

Sequence
-----------------------------------------------------------------------
Metadata:
    'description': 'M00365:23:000000000-A5A85:1:1101:19713:2393 2:N:0:0
                    orig_bc=TACCTCTCAGAA new_bc=TACCTCTCAGAA
                    bc_diffs=0'
    'id': '1AM1JR7QWMSFA_3665'
Positional metadata:
    'quality': <dtype: uint8>
Stats:
    length: 251
-----------------------------------------------------------------------
0   CCTGTTTGAT CCCCACGCTT TCGTGCCTCA GCGTCAATAA CACCATAGTA AGCTGCCTTC
60  GCAATCGGTG TTCTGTGACA TATCTATGCA TTTCACCGCT ACTTGTCACA TTCCGCCTAC
120 CTCTAGTGTA TTCAAGCTCA TCAGTATCAA GGGCACTGCG ATGGTTGAGC CACCGTCTTT
180 CACCCCTGAC TTAATAAGCC GCCTACGCAC CCTTTAAACC CAATAAATCC GGATAACGCT
240 TGGATCCTCC G

We can also view the sequence quality data for a given sequence (or sequences) by looking at the positional metadata.

In [8]:
seq1.positional_metadata.quality[:10]

0    33
1    33
2    33
3    33
4    33
5    37
6    37
7    37
8    37
9    37
Name: quality, dtype: uint8

Looking through sequence quality on a per-sequence basis is tedious and it would be diffult to decipher meaningful patterns, plotting the data is a  better solution. In order to create a meaningful plot we will first create a `pd.DataFrame` object of the quality scores. Due to limitations in data size and because a subset of our data will fairly accurately represent the quality of the full data set we will only look at athe first 500 sequences in our data set.

In [9]:
seqs = skbio.io.read(f, format='fastq', verify=False, variant='illumina1.8')

df = pd.DataFrame()
num_sequences = 500

for count, seq in enumerate(itertools.islice(seqs, num_sequences)):
    df[count] = seq.positional_metadata.quality

Now that we have a dataframe with all of our quality scores it is easy to visualize them with plotly. 

We can improve upon a basic boxplot by defining a specific color scheme. Fastq quality scores range from 0-40, and poor quality is often considered to be anything less then 20. Given this we will define a diverging colormap where an average quality score of below 20 will be a shade of red and anything above will be a shade of blue. (20 will be yellow). This will help with distinguishing regions of high versus low quality.

First define the colormap using color lover

In [10]:
purd = cl.scales['11']['div']['RdYlBu']
purd40 = cl.interp(purd, 40)

Now we can make boxplots of the quality scores on a per base basis

In [11]:
traces = []
for e in range(len(df)):
    traces.append(go.Box(
        y=df.iloc[e].values,
        name=e,
        boxpoints='none',
        whiskerwidth=0.2,
        marker=dict(
            size=.1,
            color=purd40[int(round(df.iloc[e].mean(), 0))]
        ),
        line=dict(width=1),
    ))

layout = go.Layout(
    title='Quality Score Distributions',
    yaxis=dict(
        title='Quality Score',
        autorange=True,
        showgrid=True,
        zeroline=True,
        gridcolor='#d9d4d3',
        zerolinecolor='#d9d4d3',
    ),
    xaxis=dict(
        title='Base Position',
    ),

    font=dict(family='Times New Roman', size=16, color='#2e1c18'),
    paper_bgcolor='#eCe9e9',
    plot_bgcolor='#eCe9e9'
)

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='quality-scores')


ValueError: 
    Invalid value of type 'builtins.str' received for the 'boxpoints' property of box
        Received value: 'none'

    The 'boxpoints' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['all', 'outliers', 'suspectedoutliers', False]

We now have box plots of the quality of each base for the first 500 sequences. We can see that the quality of the first ten bases is slightly lower and then increases and that the quality drops off after 225 bases. This is normal for our type of data and overall these scores are quite good meaning that we do not need to filter or trim the sequences. If the data was of very poor quality filtering or trimming would be neccessary.

## 2. Sequence Alignment
One of the most important steps in bioinformatics is sequence alignment. Aligning sequences helps us to understand the relationship between two or more sequences, it allows us to identify specific bases or regions that vary between sequences and can be used to discover novel sequences. For the purpose of this notebook we will use the global pariwise aligner from `skbio` this is known to be slow and should be updated soon. If you are aligning more than a few sequences, there are other faster aligners that would be preferrable such as [MAFFT](http://mafft.cbrc.jp/alignment/software/). [QIIME2](https://docs.qiime2.org/2.0.6/) provides a convenient set of tools that wrap aligners such as MAFFT.

### Align the first two sequences using scikit-bio
This is slow, and really only appropriate for educational purposes. In fact scikit-bio will generate a warning to this effect. You can find more information about the scikit-bio aligner in this github [issue](https://github.com/biocore/scikit-bio/issues/254). If you are aligning sequences locally scikit-bio has [optimized algorithms](http://scikit-bio.org/docs/latest/alignment.html#optimized-i-e-production-ready-alignment-algorithms) appropriate for larger scale data.

Once again we load the sequences using scikit-bio. Here we will only load the first two sequences in order to illustrate pairwise alignment.

In [12]:
seqs = [DNA(e) for e in itertools.islice(skbio.io.read(f, format='fastq', variant='illumina1.8'), 2)]

In [13]:
aligned_seqs = global_pairwise_align_nucleotide(seqs[0], seqs[1])
aligned_seqs

/home/paperspace/anaconda3/envs/bioinfo/lib/python3.6/site-packages/skbio/alignment/_pairwise.py:599: EfficiencyWarning:

You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).



(TabularMSA[DNA]
 -----------------------------------------------------------------------
 Stats:
     sequence count: 2
     position count: 255
 -----------------------------------------------------------------------
 CCTGTTTGATCCCCACGCTTTCGTGCCTCAGCG ... ATAAATCCGGATAACGCTTGGATCCTCCG----
 CCTGTTCGATCCCCACGCTTTCGTGCCTCAGCG ... ATTAAACCGGATAACGCCC---TCCTCCCCACC,
 53.0,
 [(0, 250), (0, 250)])

### Align multiple sequences
Pairwise alignment is a more simple task than multiple sequence alignment. With multiple sequences each additional sequence changes the overall alignment, meaning it must be constructed progressively with each additional sequence. This can be very computationally expensive. If you do not wish to wait for the alignment to run (or do not wish to install [An Introduction to Bioinformatics](http://readiab.org/), you can leave the following lines commented out and load the pre-aligned sequences.

In [15]:
# from iab.algorithms import progressive_msa, tree_from_distance_matrix
# from functools import partial

# f = 'run1_16s/rev_seqs/1AM1JR7QWMSFA.fastq'
# seqs = [DNA(e) for e in itertools.islice(skbio.io.read(f, format='fastq', verify=False, variant='illumina1.8'), 10)]
# seqs = [e for e in seqs if not e.has_degenerates()]
# msa = progressive_msa(seqs, global_pairwise_align_nucleotide)
# msa.write('msa10.fna')
msa = skbio.alignment.TabularMSA.read('/home/paperspace/data/biology/bioinfo/msa10.fna', constructor=DNA)

In order to create a meaningful visualization of the multiple sequence alignment we will use heatmap functionality of plotly.  
First we assign a numeric value to each possible character in our alignment, "A", "T", "G", "C", and "-"

In [16]:
base_dic = {'A': 1, 'C': .25, 'G': .5, 'T': .75, '-': 0}

Next we define a function that takes an alignment and returns two, two-dimensional arrays, one of the characters in the alignment and one of the numeric value that represents the character defined in the dictionary above. The numeric value will be used to define the color in the heatmap. The function below will create an array of the alignment such the only bases that are colored differently are bases that are *different* from the first sequence. This will make identifying differences in sequences much easier. If each based was given a unique color regardless of it's relationship to other sequences the plot would be noisy and difficult to interpret.

In [17]:
def seq_align_for_plot(msa):
    base_text = [list(str(e)) for e in msa]
    base_values = np.zeros((len(base_text), len(base_text[0])))
    for i in range(len(base_text[0])):
        for j in range(len(base_text)):
            if base_text[j][i] != base_text[0][i]:
                base_values[j][i] = base_dic[base_text[j][i]]
    return(base_text, base_values)

base_text, base_values = seq_align_for_plot(msa)

Define a colorscale where the values for each base is given a defined color

In [18]:
colorscale=[[0.00, '#F4F0E4'], 
            [0.25, '#1b9e77'], 
            [0.50, '#d95f02'], 
            [0.75, '#7570b3'],
            [1.00, '#e7298a']]

Create a list of arbitrary sequences names (The original sequence names were randomly generated and do not have meaning associated with them).

In [19]:
seq_names = ["Seq " + str(e + 1) for e in range(len(base_text))]

Finally we can plot the alignment.

In [32]:
fig = FF.create_annotated_heatmap(base_values, 
                                  annotation_text=base_text, 
                                  colorscale=colorscale)

fig['layout'].update(
    title="Aligned Sequences",
    xaxis=dict(ticks='', 
               side='top',
               ticktext=list(np.arange(0, len(base_text[0]), 10)),
               tickvals=list(np.arange(0, len(base_text[0]), 10)),
               showticklabels=True,
               tickfont=dict(family='Bookman', 
                             size=18, 
                             color='#22293B',
                            ),
              ),
    
    yaxis=dict(autorange='reversed',
               ticks='', 
               ticksuffix='  ',
               ticktext=seq_names,
               tickvals=list(np.arange(0, len(base_text))),
               showticklabels=True,
               tickfont=dict(family='Bookman', 
                         size=18, 
                         color='22293B',
                            ),
              ),
    width=10000,
    height=450,
    autosize=True,
    annotations=dict(font=dict(family='Courier New, monospace',
                                size=14,
                                color='#3f566d'
                               ),
                      )
)
iplot(fig, filename='msa')

/home/paperspace/anaconda3/envs/bioinfo/lib/python3.6/site-packages/plotly/tools.py:1545: UserWarning:

plotly.tools.FigureFactory.create_annotated_heatmap is deprecated. Use plotly.figure_factory.create_annotated_heatmap



ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of layout.yaxis.tickfont
        Received value: '22293B'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, 

The above plot represents a global multiple sequence alignment. Bases that differ from the first sequence are colored uniquely so that differences can be easily identified. The numbers above the plot represent the position in the alignmment.

## Tree Building

Another common task in bionformatics is phylogenetic reconstruction. In order to illustrate this we are going to compare the distance between sequences using hamming distance (the number of absolute differences between a pair of sequences. The scikit-bio implementation gives the ratio of the differences to the length of the sequences. There are many ways to measure distance between two pairs of sequences and any function that takes two arguments and returns a float representing the distance between them may be used.

Using scikit-bio we first create a distance matrix between all pairs of sequences. Because we are using hamming we will use the already aligned sequences to create our distance matrix. 

In [26]:
from skbio.sequence.distance import hamming
from skbio.stats.distance import DistanceMatrix
hamming_dm = DistanceMatrix.from_iterable(msa, metric=hamming, key='id')

Now that we have a distance matrix of the sequences we could quickly and easily use the heatmap functionality of plotly to visualize out data.

In [27]:
colorscale = [[0.0,'#4bd4d1'], [0.5, '#1c9099']]  # custom colorscale
['#ece2f0','#a6bddb','#1c9099']
trace = go.Heatmap(x=seq_names, y=seq_names, z=hamming_dm.data, colorscale='Viridis')

fig = go.Figure(data=[trace])

fig['layout'].update(
    xaxis=dict(ticks=''),
    yaxis=dict(ticks='', ticksuffix='  '),
    width=700,
    height=700,
    autosize=False,
    margin=go.Margin(
        l=200,
        b=200,
        pad=4
    )
)
iplot(fig, filename='dm_heatmap')

/home/paperspace/anaconda3/envs/bioinfo/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:396: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




The heat map while visualy appealing does not reveal the structure or our data as well as we might hope. A better representation can be given with a dendrogram.

### Note: 
Due to the nature of the distance calculations some of the sequenes were calculated to have a distance of 0 from each other. We can see from the multiple sequence alignment above that there are no sequences that are identical (if they were identical we would want to collapse them into a single tip in the tree). Because we can't have a branch length of 0 we will add a small amount to all of the distances, which will give a visually more accurate tree.

In [28]:
hamming_array = hamming_dm.data + .001
np.fill_diagonal(hamming_array, 0)
names = hamming_dm.ids

Now we can plot our data using the dendrogram functionality of plotly. The plot below makes the assumption (for illustrative purposes) that all sequences belong to one of two arbitrarily defined groups. In reality the groups may be related to the origin of the sequences (i.e., they came from different samples). We will also color any node that does not lead to a terminal node a different color.

In [29]:
group1 = ['Seq 1', 'Seq 7', 'Seq 9', 'Seq 3']
group2 = ['Seq 2', 'Seq 4', 'Seq 5', 'Seq 6', 'Seq 8']

In [31]:
names = ["Seq " + str(e + 1) for e in range(len(hamming_array))]
fig = FF.create_dendrogram(hamming_array.data, 
                           orientation='right',
                           labels=names)


fig['layout'].update(
    width=500,
    height=700,                     
)

fig['layout']['yaxis'].update(dict(side='right', 
                                   showline=False,
                                   ticks='',
                                   zeroline=True,
                                   zerolinewidth=4,
                                   zerolinecolor='#969696')
                              )

fig['layout']['xaxis'].update(dict(showline=False,
                                   showticklabels=True,
                                   ticktext=np.array(['0.0', '0.2', '0.4', '0.6', '0.8']),
                                   tickvals=[-0.8, -0.6, -0.4, -0.2, -0.0],
                                   tickfont=dict(color='#969696'),
                                   mirror=None,
                                   ticklen=8,
                                   tickwidth=4,
                                   tickcolor='#969696',
                                   range=[-0.9, 0.1])
                             )
tips = []
for i in fig['data']:
    if 0 in i['x']:
        i['marker']['color'] = '#ffbe4f'
    else:
        i['marker']['color'] = '#0c457d'

for i in range(len(names)):
    if fig['layout']['yaxis']['ticktext'][i] in group1:
        color = '#e8702a'
        symbol = 'circle'
    else:
        color = '#0ea7b5'
        symbol = 'triangle-left'
        
    x = [-0.0]
    y = [fig['layout']['yaxis']['tickvals'][i]]
    trace = go.Scatter(x=x, y=y, 
                       mode='markers',
                       marker=dict(
                            size=10,
                            color=color,
                            symbol=symbol)
                      )
    list(fig['data']).append(trace)

iplot(fig, filename='phylo-dendrogram')

The above plot gives a much better interpretation of the relationship between pairs and groups of sequences when compared to the heatmap. Being able to quickly identify groups of sequences also helps to determine the structure of our data set. 